In [1]:
import numpy as np
import pandas as pd
import sympy as sp

In [2]:
df = pd.read_excel('Teste.xlsx',header=None)
df
# número de nós
# ID / x / y / restrição em x / restrição em y / restrição momento
# número de materiais
# ID / módulo de elasticidade
# número de barras
# ID / nó inicial / nó final / secão x / seção y / IDmaterial
# número de cargas em nós
# nó / valor da carga x / valor da carga y
# número de cargas em barras
# barra / direção / inicio da carga / fim da carga / valor da carga no inicio / valor da carga no fim

,0,1,2,3,4,5
0,3,NaN,NaN,NaN,NaN,NaN
1,1,0.0,0.0,1.0,1.0,1.0
2,2,0.0,4.0,0.0,0.0,0.0
3,3,6.0,4.0,1.0,1.0,1.0
4,1,NaN,NaN,NaN,NaN,NaN
5,1,1.0,NaN,NaN,NaN,NaN
6,2,NaN,NaN,NaN,NaN,NaN
7,1,1.0,2.0,1.0,1.0,1.0
8,2,2.0,3.0,1.0,1.0,1.0
9,2,NaN,NaN,NaN,NaN,NaN


In [30]:


class No:
    def __init__(self, x, y, Rx, Ry, Rm):

        self.x = x          #posição x do nó
        self.y = y          #posição y do nó
        self.Rx = Rx        #apoio na direção x (0 sem apoio, 1 com apoio)
        self.Ry = Ry        #apoio na direção y (0 sem apoio, 1 com apoio)
        self.Rm = Rm        #apoio em relação ao momento (0 sem apoio, 1 com apoio)
        self.carga = [0,0,0]

    def adicionarcarga(self,carga):
      self.carga[0] = carga.forcax
      self.carga[1] = carga.forcay
      self.carga[2] = carga.forcamomento

class Material:
    def __init__(self, E):
      self.E = E            #módulo de elasticidade


class Barra:
    def __init__(self, no1, no2, secaox, secaoy):

      self.no1 = no1
      self.no2 = no2
      self.secaox = secaox
      self.secaoy = secaoy
      self.L = calculacomprimento(no1,no2)
      self.I = calculainercia(secaox,secaoy)
      self.Area = calculaarea(secaox,secaoy)


class Carga:
    def __init__(self, no, forcax, forcay, forcamomento):
      self.forcax = forcax
      self.forcay = forcay
      self.forca = np.sqrt(pow(forcax,2) + pow(forcay,2))
      self.forcamomento = forcamomento
      self.no = no
      self.posicaox = no.x  #é passada a posição da carga através da posição do nó onde a carga foi aplicada
      self.posicaoy = no.y   # é necessário atualizar para possibilitar a utilização de outros tipos de carga,
                             # como carga distribuída, ou uma carag não aplicada em um nó



def calculaliberdade(Nos):
  grauliberdade = 0
  for no in Nos:
    if no.Rx == 1:
      grauliberdade += 1
    if no.Ry == 1:
      grauliberdade += 1
    if no.Rm == 1:
      grauliberdade += 1

  return grauliberdade - 3



def calculaarea(secaox,secaoy):
  return secaox*secaoy



def calculainercia(secaox,secaoy):
  return secaox*(secaoy**3)/12



def calculacomprimento(no1,no2):
  np.sqrt(pow(abs(no1.x - no2.x),2) + pow(abs(no1.y - no2.y),2))

In [39]:
class estrutura:
	def __init__(self, df):
		self.nos = {}
		self.barras = {}
		self.materiais = {}

		# leitura dos nós
		pos = 0
		num_nos = df.iloc[pos, 0]
		pos = pos + 1

		for i in range(1, num_nos+1):
			self.nos[df.iloc[pos, 0]] = No(df.iloc[pos, 1], df.iloc[pos, 2], df.iloc[pos, 3], df.iloc[pos, 4], df.iloc[pos, 5])
			pos += 1

		# leitura dos materiais
		num_materiais = df.iloc[pos, 0]
		pos = pos + 1
		self.materiais[df.iloc[pos, 0]] = Material(df.iloc[pos, 1])
		pos += 1
		


		# leitura das barras
		num_barras = df.iloc[pos, 0]
		pos = pos + 1

		for i in range(1, num_barras+1):
			self.barras[df.iloc[pos, 0]] = Barra(self.nos[df.iloc[pos, 1]], self.nos[df.iloc[pos, 2]], df.iloc[pos, 3], df.iloc[pos, 4])
			pos += 1

		# leitura das cargas nos nós
		num_cargas = df.iloc[pos, 0]
		pos = pos + 1

		for i in range(1, num_cargas+1):
			x = Carga(self.nos[df.iloc[pos, 0]], df.iloc[pos, 1], df.iloc[pos, 2], df.iloc[pos, 3])	
			self.nos[df.iloc[pos, 0]].adicionarcarga(x)
			
			

			pos += 1




In [40]:
struct = estrutura(df)

print(struct)

TypeError: Carga.__init__() missing 1 required positional argument: 'forcamomento'

In [3]:
def rotacionamatriz(x1,x2,y1,y2,theta):
  x = x2 - x1
  y = y2 - y1
  A = sp.zeros(1,2)
  A[0,0] = x
  A[0,1] = y
  A
  rotacao = sp.zeros(2,2)
  rotacao[0,0] = sp.cos(theta)
  rotacao[0,1] = -sp.sin(theta)
  rotacao[1,0] = sp.sin(theta)
  rotacao[0,0] = sp.cos(theta)

  B = A*rotacao
  return B

In [5]:
fodase = rotacionamatriz(0,0,0,4,270)
fodase

Matrix([[4*sin(270), 0]])